In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
#tf.enable_eager_execution()
tf.__version__

'1.14.0'

In [2]:
import os
import random
from sklearn.model_selection import train_test_split
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
DATA_DIR = './data/'
model_name = model_name = 'efficientnet-b0'
ckpt_dir = './efficientnet-b0/'
export_ckpt_dir = None
BATCH_SIZE = 32
#MEAN_RGB = [0.485 * 255, 0.456 * 255, 0.406 * 255]
#STDDEV_RGB = [0.229 * 255, 0.224 * 255, 0.225 * 255]

In [4]:
all_image_paths = [DATA_DIR+i for i in os.listdir(DATA_DIR)]
random.shuffle(all_image_paths)

In [5]:
all_image_labels=[]
for i in all_image_paths:
    if 'dog' in i:
        all_image_labels.append(1)
    else:
        all_image_labels.append(0)

In [6]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    #image -= tf.constant(MEAN_RGB, shape=[1, 1, 3], dtype=image.dtype)
    #image /= tf.constant(STDDEV_RGB, shape=[1, 1, 3], dtype=image.dtype)
    return image

In [7]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [8]:
train_paths, test_paths, train_labels, test_labels = train_test_split(all_image_paths, all_image_labels, test_size=0.2, random_state=10)
tra_paths,val_paths,tra_labels,val_labels = train_test_split(train_paths,train_labels,test_size=0.2,random_state=10)

In [9]:
def build_dataset(images,labels):
    ds = tf.data.Dataset.from_tensor_slices((tra_paths, tra_labels))
    # The tuples are unpacked into the positional arguments of the mapped function
    def load_and_preprocess_from_path_label(path, label):
        return load_and_preprocess_image(path), label
    image_label_ds = ds.map(load_and_preprocess_from_path_label)
    ds = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(labels)))
    ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    return ds

In [10]:
train_batches = build_dataset(tra_paths,tra_labels)
test_batches = build_dataset(test_paths,test_labels)
val_batches = build_dataset(val_paths,val_labels)

W0711 19:28:16.131233 10612 deprecation.py:323] From <ipython-input-9-07c1cf0db198>:7: shuffle_and_repeat (from tensorflow.python.data.experimental.ops.shuffle_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [ ]:
#image_batch, label_batch = next(iter(ds))

In [11]:
import efficientnet_model
import efficientnet_builder

W0711 19:28:19.214233 10612 deprecation_wrapper.py:119] From F:\jupyter\efficientnet\utils.py:86: The name tf.layers.BatchNormalization is deprecated. Please use tf.compat.v1.layers.BatchNormalization instead.

W0711 19:28:19.217235 10612 deprecation_wrapper.py:119] From F:\jupyter\efficientnet\utils.py:200: The name tf.layers.Layer is deprecated. Please use tf.compat.v1.layers.Layer instead.



In [12]:
blocks_args, global_params = efficientnet_builder.get_model_params(model_name, None)
input = tf.keras.Input(shape=(224,224,3))
basemodel = efficientnet_model.Model(blocks_args, global_params)
with tf.variable_scope(model_name):
    features = basemodel(input, training=True, features_only=True)
    tf.train.init_from_checkpoint('./efficientnet-b0/model.ckpt', {'efficientnet-b0/':'efficientnet-b0/'})

W0711 19:28:19.249234 10612 deprecation_wrapper.py:119] From F:\jupyter\efficientnet\efficientnet_builder.py:157: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0711 19:28:19.261233 10612 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0711 19:28:19.264234 10612 deprecation_wrapper.py:119] From F:\jupyter\efficientnet\efficientnet_model.py:201: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W0711 19:28:19.309233 10612 deprecation_wrapper.py:119] From F:\jupyter\efficientnet\efficientnet_model.py:372: The name tf.layers.Dense is deprecated. Please use tf.compat.v1.layers.Dense instead.

W0711 19:28:19.520233

In [13]:
base_model = tf.keras.Model(inputs=input,outputs=features)
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

W0711 19:28:28.979298 10612 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
initial_epochs = 10
steps_per_epoch = round(len(tra_labels))//BATCH_SIZE
validation_steps = round(len(val_labels))//BATCH_SIZE
print(steps_per_epoch)

500


In [16]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data = val_batches,
                    validation_steps = validation_steps )

Epoch 1/10
500/500 [==============================] - 439s 878ms/step - loss: 0.2103 - acc: 0.9108 - val_loss: 0.1265 - val_acc: 0.9515
Epoch 2/10
500/500 [==============================] - 244s 489ms/step - loss: 0.1292 - acc: 0.9467 - val_loss: 0.1036 - val_acc: 0.9580
Epoch 3/10
500/500 [==============================] - 244s 489ms/step - loss: 0.1235 - acc: 0.9513 - val_loss: 0.1022 - val_acc: 0.9595
Epoch 4/10
500/500 [==============================] - 247s 493ms/step - loss: 0.1095 - acc: 0.9553 - val_loss: 0.0941 - val_acc: 0.9603
Epoch 5/10
500/500 [==============================] - 246s 492ms/step - loss: 0.1112 - acc: 0.9555 - val_loss: 0.0923 - val_acc: 0.9668
Epoch 6/10
500/500 [==============================] - 288s 577ms/step - loss: 0.1043 - acc: 0.9578 - val_loss: 0.0833 - val_acc: 0.9678
Epoch 7/10
500/500 [==============================] - 247s 494ms/step - loss: 0.0960 - acc: 0.9620 - val_loss: 0.1013 - val_acc: 0.9615
Epoch 8/10
500/500 [============================

In [17]:
OUTPUT = 'output'

if not os.path.exists(OUTPUT):
    os.mkdir(OUTPUT)

model.save_weights(os.path.join(OUTPUT, 'b0.h5'))

In [18]:
steps = round(len(test_labels))//BATCH_SIZE

In [19]:
model.evaluate(test_batches,steps=steps)

156/156 [==============================] - 120s 768ms/step - loss: 0.0665 - acc: 0.9750


[0.0664522725945482, 0.9749599]